In [1]:
# MIT image
# Made by : HERO Kwon
# Date : 190108

import os
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
import math
import cmath

# data path
#path_meta = '/home/herokwon/mount_data/Data/Wi-Fi_meta/'
#path_csi = '/home/herokwon/mount_data/Data/Wi-Fi_processed/'
#path_csi_np = '/home/herokwon/mount_data/Data/Wi-Fi_processed_npy/'

# data path_mi
path_csi = 'J:\\Data\\Wi-Fi_processed\\'
path_csi_np = 'J:\\Data\\Wi-Fi_processed_npy\\'
path_meta = 'J:\\Data\\Wi-Fi_meta\\'
path_sc = 'J:\\Data\\Wi-Fi_info\\'
path_mit_image = 'J:\\Data\\Wi-Fi_mit_image\\'

# data info
df_info = pd.read_csv(path_meta+'data_subc_sig_v1.csv')
df_info = df_info[df_info.id_location==1]

#df_info = df_info.iloc[0:250]
#df_info = df_info.iloc[250:500]
#df_info = df_info.iloc[500:750]
#df_info = df_info.iloc[750:1000]
#df_info = df_info.iloc[1000:1250]
#df_info = df_info.iloc[1250:1500]
#df_info = df_info.iloc[1500:1750]
#df_info = df_info.iloc[1750:2000]
#df_info = df_info.iloc[2000:2250]
#df_info = df_info.iloc[2250:2500]
#df_info = df_info.iloc[2500:2750]
#df_info = df_info.iloc[2750:3000]
#df_info = df_info.iloc[3000:3250]
df_info = df_info.iloc[3250:3500]
#df_info = df_info.iloc[3500:3632]

person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))
csi_time = 50 #15000 #int(np.max(df_info['len']))
# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

# freq BW list
bw_list = pd.read_csv(path_meta+'wifi_f_bw_list.csv')

# avg Array
with open(path_meta + 'dict_avgcsi.pickle','rb') as f:
    dict_avg = pickle.load(f)

# 3D scan param
m,n = 2,3
c =  299792458 # speed of light 
#r = (160 + 160 + 164) * 0.01 # meter
r = 1.64 #meter
d = 45 * 0.01 # meter
ch = 8#3
max_subc = 30


# make data generator
def gen_csi(df_info,dict_avg,id_num,len_num):
    for file in np.unique(df_info.id.values):
        
        # Label
        id_key = df_info[df_info.id==file][['id_person','id_location','id_direction','id_exp']].values[0].astype('int')
        data1_y = dict_id[id_key[0]]
        
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
            
        # subtract avg
        read_avg = dict_avg[id_key[0]][0]
        nonzero_avg = read_avg[np.nonzero(read_avg)].reshape([-1,30,2,3])
        arr_avg = np.mean(nonzero_avg,axis=0)
        data1_diff = data1 - arr_avg #np.diff(data1,axis=0)

        s_idx = (np.arange(len_num) * data1_diff.shape[0] / len_num).astype('int')
        data1_s = data1_diff[s_idx]
        # zero pad
        #pad_len = len_num - data1_diff.shape[0]
        #data1_pad = np.pad(data1_diff,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)

        # subcarrier info
        #data1_sc_df = pd.read_csv(path_sc + file + '_df_sc.csv')
        #data1_time = np.diff(data1_sc_df['timestamp_low'])
        #data1_time_pad = np.pad(data1_time,((1,pad_len)),'constant',constant_values=0)
        
        
        yield(data1_s ,data1_y,id_key,file)#,data1_time_pad)

gen = gen_csi(df_info,dict_avg,no_classes,csi_time)



# 3D Reconstruction func
from numba import vectorize
@vectorize(['complex64(complex64,float32,float32,int32,int32,float32,float32,float32,int32)'], target='cpu')
def Recon3d(sig,theta,sigma,m,n,lam,d,r,c):
    #above_eq1 = 1j * (2*math.pi) * k * r * t / c
    #above_eq1 = 1j * (2*math.pi) * c * dt * 0.000001 / lam
    above_eq1 = 1j * (2*math.pi) * r / lam
    above_eq2 = 1j * (2*math.pi/lam) * math.sin(theta) * ((n+1)*d*math.cos(sigma) + (m+1)*d*math.sin(sigma))
    eq_res = sig* cmath.exp(above_eq1) * cmath.exp(above_eq2)
    #eq_res = cmath.exp(above_eq1) * cmath.exp(above_eq2)
    return eq_res #math.atan2(eq_res.imag,eq_res.real)


def Calc3d(target_sig,max_subc,theta,sigma,bw_list,ch,d,r,c,m,n):
    #vectorize arrays
    subc_array = np.ones([csi_time,max_subc,m,n])
    m_array = np.ones([csi_time,max_subc,m,n])
    n_array = np.ones([csi_time,max_subc,m,n])
    for i in range(max_subc):
        subc_array[:,i,:,:] = i
    for i in range(m):
        m_array[:,:,i,:] = i
    for i in range(n):
        n_array[:,:,:,i] = i
    subc_array = subc_array.flatten().astype(np.int32)
    m_array = m_array.flatten().astype(np.int32)
    n_array = n_array.flatten().astype(np.int32)
    lam_array = np.array([c*0.000001/bw_list[str(e)][ch] for e in subc_array],dtype=np.float32)
    
    target_array = target_sig.flatten().astype(np.complex64)
    
    #3d recon
    sum_eq = np.zeros(csi_time*max_subc*m*n*th_range,dtype=np.complex_)
    sum_eq = Recon3d(target_array,theta,sigma,m_array,n_array,lam_array,d,r,c)

    sum_reshape = np.sum(sum_eq.reshape([-1,max_subc,m,n]),axis=(1,2,3))
    return(np.abs(sum_reshape))


for f in np.unique(df_info.id.values):

    target_sig,target_lab,target_id,target_file = next(gen)
    th_range,si_range = (10,10)
    sig_mat = np.zeros([csi_time,2*th_range,2*si_range])

    import time
    t1 = time.time()
    for idx_th,i in enumerate(range(-th_range,th_range)):
        for idx_si,j in enumerate(range(-si_range,si_range)):
            theta = i * (np.radians(60)/2) / th_range
            sigma = j * (np.radians(60)/2) / si_range
            sig_mat[:,idx_th,idx_si] = Calc3d(target_sig,max_subc,theta,sigma,bw_list,ch,d,r,c,m,n)
    print(time.time()-t1)
    print(f)

    np.save(path_mit_image + f + '.npy',sig_mat)

#th_array = (np.radians(30)/2)/th_range * np.arange(-th_range,th_range)
#si_array = (np.radians(30)/2)/si_range * np.arange(-si_range,si_range)

63.48074460029602
S031_1_2_6
65.55556130409241
S031_1_2_7
62.15650200843811
S031_1_2_8
64.51715397834778
S031_1_2_9
62.4983069896698
S031_1_3_1
62.51329827308655
S031_1_3_12
62.0835440158844
S031_1_3_2
63.94548058509827
S031_1_3_3
62.653217792510986
S031_1_3_4
61.518866777420044
S031_1_3_5
66.43106079101562
S031_1_3_6
62.24045395851135
S031_1_3_7
63.235886335372925
S031_1_3_8
62.108530044555664
S031_1_4_1
58.69248104095459
S031_1_4_10
53.392507553100586
S031_1_4_11
56.234885692596436
S031_1_4_12
53.84225082397461
S031_1_4_13
57.640082120895386
S031_1_4_14
54.46889281272888
S031_1_4_3
54.76272439956665
S031_1_4_4
46.20161533355713
S031_1_4_5
44.88736391067505
S031_1_4_6
45.56397891044617
S031_1_4_7
47.351956844329834
S031_1_4_8
45.346102476119995
S031_1_4_9
44.95532512664795
S032_1_1_1
46.401498317718506
S032_1_1_10
45.53399705886841
S032_1_1_11
46.8602397441864
S032_1_1_12
47.312978982925415
S032_1_1_2
48.50130081176758
S032_1_1_3
46.34853219985962
S032_1_1_4
45.285135984420776
S032_1_